In [1]:
spark

In [5]:
import gc

In [2]:
sampleData = '../../datasets/krk/krk'
trainFilename = sampleData + '.data'
namesFilename = sampleData + '.names'

In [3]:
!ls -al

total 112
drwxr-xr-x.  6 nkamat nkamat  4096 Jul 27 10:49 .
drwxrwxr-x.  8 nkamat nkamat  4096 Jul 27 10:49 ..
-rwxr-xr-x.  1 nkamat nkamat   519 Jul 27 10:49 combineModels.py
drwxr-xr-x. 49 nkamat nkamat  4096 Jul 27 10:49 data
-rw-r--r--.  1 nkamat nkamat  8013 Jul 27 10:49 dataset.py
-rw-r--r--.  1 nkamat nkamat   722 Jul 27 10:49 derby.log
-rw-r--r--.  1 nkamat nkamat  2061 Jul 27 10:49 dummyEncoder.py
-rwxr-xr-x.  1 nkamat nkamat  2004 Jul 27 10:49 genesis.py
drwxr-xr-x.  8 nkamat nkamat  4096 Jul 27 10:49 .git
-rw-r--r--.  1 nkamat nkamat    23 Jul 27 10:49 .gitignore
drwxr-xr-x.  2 nkamat nkamat  4096 Jul 27 10:49 .ipynb_checkpoints
-rw-r--r--.  1 nkamat nkamat 33234 Jul 27 10:49 learningModel.py
drwxr-xr-x.  5 nkamat nkamat  4096 Jul 27 10:49 metastore_db
-rw-r--r--.  1 nkamat nkamat  7741 Jul 27 10:49 names.py
-rw-r--r--.  1 nkamat nkamat  6376 Jul 27 10:49 options.py
-rw-r--r--.  1 nkamat nkamat   367 Jul 27 10:49 README
-rw-r--r--.  1 nkamat nkamat    72 Jul 27 10:49 Untitle

In [45]:
import numpy as np
from dataset import dataset
from options import getOptions
from learningModel import learningModel

trainingSet = dataset(trainFilename, namesFilename)

In [46]:
trainingSet.data.head()

,white king file (col),white king rank (row),white rook file,white rook rank,black king file,blank king rank
0,a,1,b,3,c,2
1,a,1,c,1,c,2
2,a,1,c,1,d,1
3,a,1,c,1,d,2
4,a,1,c,2,c,1


In [20]:
import pickle
import gzip
import sys
import numpy as np
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
from sklearn.svm import SVC, SVR
from sklearn.svm import NuSVC, NuSVR
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import StratifiedKFold, LeaveOneOut, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, explained_variance_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import recall_score, precision_score, f1_score, log_loss
from sklearn.preprocessing import Imputer
# from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from dummyEncoder import dummyEncoder

classifier = GradientBoostingClassifier(n_estimators=1, \
#                                    nthread=-1, \
                                   random_state=None, \
                                   max_depth=None)

model = Pipeline([('data', dummyEncoder()), \
                  ('imputer', Imputer(missing_values='NaN', strategy='median', axis=0, verbose=100)), \
                  ('classifier', classifier)])

In [21]:
kFold = KFold(n_splits=2, shuffle=True, random_state=None)
groundTruth = trainingSet.classes
datasetPredictions = [None] * len(groundTruth)
for trainIndices, testIndices in kFold.split(trainingSet.data, trainingSet.classes):
    trainX = trainingSet.data.ix[trainIndices, :]
    trainY = trainingSet.classes[trainIndices]
    testX = trainingSet.data.ix[testIndices, :]
    testY = trainingSet.classes[testIndices]

    # Learn the model
    model.fit(trainX, trainY)
    
    foldPredictions = model.predict(testX)
    
    for i, v in enumerate(testIndices):
        datasetPredictions[v] = foldPredictions[i]
    

In [30]:
import pandas as pd
new_df = pd.concat((trainingSet.data, trainingSet.classes), axis=1)
new_df.head()

,white king file (col),white king rank (row),white rook file,white rook rank,black king file,blank king rank,target
0,a,1,b,3,c,2,draw
1,a,1,c,1,c,2,draw
2,a,1,c,1,d,1,draw
3,a,1,c,1,d,2,draw
4,a,1,c,2,c,1,draw


In [43]:
type(trainingSet.classes)

pandas.core.series.Series

In [140]:
de = dummyEncoder()
temp = de.fit_transform(trainingSet.data)
temp = map(lambda arr: Vectors.dense(arr), temp)
data = spark.createDataFrame(zip(trainingSet.classes, temp),schema=["label", "features"])

In [156]:
# mydf.show()
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(data)
(trainingData, testData) = data.randomSplit([0.8, 0.2])
rf = RandomForestClassifier(labelCol="indexedLabel", \
                            featuresCol="features", \
                            numTrees=100, \
                            featureSubsetStrategy='onethird',
                            maxDepth=7,
                            impurity='entropy')
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

pipeline = Pipeline(stages=[featureIndexer, labelIndexer, rf, labelConverter])
# pipeline = Pipeline(stages=[featureIndexer, rf])


# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1035, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: org does not exist in the JVM

In [124]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="indexed", featuresCol="features", numTrees=100)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)
model = rf.fit()

In [102]:
[type(ii) for ii in temp]

[]

In [86]:
from pyspark.ml.linalg import Vectors

# dff = map(lambda x: Vectors.dense(x[:]), temp)
dff = spark.sparkContext.parallelize(temp)
dff.take(5)
dff = dff.map(lambda x: )
# dff.first() == temp[0]
mydf = spark.createDataFrame(dff,schema=["features"])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [59]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, '../../jupyter-notebooks/datasets/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])


In [65]:
testData.map(lambda x: x.label).take(5)

[0.0, 1.0, 1.0, 1.0, 0.0]

In [87]:
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz", "kddcup.data.gz")
data_file = "./kddcup.data.gz"
raw_data = sc.textFile(data_file)
print("Train data size is {}".format(raw_data.count()))
ft = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz", "corrected.gz")
test_data_file = "./corrected.gz"
test_raw_data = sc.textFile(test_data_file)
print ("Test data size is {}".format(test_raw_data.count()))

<SparkContext master=local[*] appName=PySparkShell>

In [138]:
from pyspark.ml.feature import VectorIndexer

data = spark.read.format("libsvm").load("../../jupyter-notebooks/datasets/sample_libsvm_data.txt")

indexer = VectorIndexer(inputCol="features", outputCol="indexed", maxCategories=10)
indexerModel = indexer.fit(data)

categoricalFeatures = indexerModel.categoryMaps
print("Chose %d categorical features: %s" %
      (len(categoricalFeatures), ", ".join(str(k) for k in categoricalFeatures.keys())))

# Create new column "indexed" with categorical values transformed to indices
indexedData = indexerModel.transform(data)
indexedData.show()

Chose 351 categorical features: 645, 69, 365, 138, 101, 479, 333, 249, 0, 555, 666, 88, 170, 115, 276, 308, 5, 449, 120, 247, 614, 677, 202, 10, 56, 533, 142, 500, 340, 670, 174, 42, 417, 24, 37, 25, 257, 389, 52, 14, 504, 110, 587, 619, 196, 559, 638, 20, 421, 46, 93, 284, 228, 448, 57, 78, 29, 475, 164, 591, 646, 253, 106, 121, 84, 480, 147, 280, 61, 221, 396, 89, 133, 116, 1, 507, 312, 74, 307, 452, 6, 248, 60, 117, 678, 529, 85, 201, 220, 366, 534, 102, 334, 28, 38, 561, 392, 70, 424, 192, 21, 137, 165, 33, 92, 229, 252, 197, 361, 65, 97, 665, 583, 285, 224, 650, 615, 9, 53, 169, 593, 141, 610, 420, 109, 256, 225, 339, 77, 193, 669, 476, 642, 637, 590, 679, 96, 393, 647, 173, 13, 41, 503, 134, 73, 105, 2, 508, 311, 558, 674, 530, 586, 618, 166, 32, 34, 148, 45, 161, 279, 64, 689, 17, 149, 584, 562, 176, 423, 191, 22, 44, 59, 118, 281, 27, 641, 71, 391, 12, 445, 54, 313, 611, 144, 49, 335, 86, 672, 172, 113, 681, 219, 419, 81, 230, 362, 451, 76, 7, 39, 649, 98, 616, 477, 367, 535, 1

In [110]:
gc.collect()

62357